In [ ]:
%run config.ipynb

In [ ]:
from cortex import Cortex, Message
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import sys
import json
import re
import time
import datetime
import dill
import sklearn
import sklearn2

client = Cortex.client()
exp = client.experiment('<namespace>/<namefrom_previous_notebook>')

exp

In [ ]:
run = exp.last_run() 

#to select a specific model you want uncomment the following and comment out above then put the exp id you want from above.
#run = exp.get_run(<ID>)

model = run.get_artifact('model')

print('Run id: %s \nModel name: "%s"' %(run.id, model))

In [ ]:
builder = client.builder()
builder.action('flowers/iris-predict')\
        .with_requirements(['tensorflow>=1.12,<2',
                            'keras>=2.2.4,<3'])\
        .from_model(model, model_type='tensorflow')\
        .daemon()\
        .build()

The following cell checks the status of the action created above.  It may take a few moments for the Deployment status to change from **STARTED** to  **COMPLETED(running)**

In [ ]:
action = client.action('flowers/iris-predict')
action

In [ ]:
%%time

#This may take some time - if you see a "No load balancer found for invoked daemon.  Stack may still be initializing" error wait a few moments and try again

params = {'columns': ['sepal_len','sepal_w','petal_len','petal_w'],
           'values': [4.9,3.1,1.5,0.2]}
frame = pd.DataFrame(params)
result = action.invoke(message=client.message(params))
print(result.payload)

In [ ]:
x = np.argmax(result.payload['values'][0])

iris_dict = {0:'Iris-setosa',1:'Iris-versicolor',2:'Iris-virginica'}

iris_dict[x]

**Create Skill Using the Action**

To create a skill from the action, call the builder to get a skill builder.

In [ ]:
skill_builder = builder.skill('flowers/iris-predict')\
     .title('Iris Flower Prediction')\
     .description('Measurement based prediction for the type of Iris flowers.')

A resulting schema is used by the skill builder to identify elements of the input. In this example, the schema describes the measurement values for parts of the Iris flower.

In [ ]:
schema_name = 'flowers/iris-input-schema'

builder.schema(schema_name)\
     .title('Iris Prediction Schema Instance')\
     .from_parameters([{'name': 'values', 'type': 'array', 'format': 'float32'}]).build()

skill_builder = skill_builder.input('iris-measurement').title('Iris Measurements')\
     .use_schema(schema_name)\
     .all_routing(action, 'measurement-based-prediction').build()

Specify the skill construction. The construction describes the skill including a name that matches the action name and a description of the output.

In [ ]:
skill_builder = skill_builder.output('measurement-based-prediction').title('Classification')\
    .parameter(name='IrisClass', type='array', format='float32').build()

With the skill builder fully specified, call to_camel so the skill’s contents can be reviewed.

In [ ]:
skill_builder.to_camel()

**Publish the Skill**

Invoke the builder to publish the skill. Upon publication, the skill can be used in Cortex Studio with other skills to compose agents.

In [ ]:
skill = skill_builder.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

# Test the skill